<a href="https://colab.research.google.com/github/Aneesha0306/LegalEase/blob/main/LegalEase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate

In [ ]:
import pandas as pd
import re
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving text_summary_pair.csv to text_summary_pair (2).csv


In [ ]:
df = pd.read_csv(list(uploaded.keys())[0], encoding='latin1')

In [ ]:
def clean_legal_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s+([.,;:!?])', r'\1', text)
    text = re.sub(r'([.,;:!?])\s+', r'\1 ', text)
    text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
    return text.strip()

In [ ]:
def legal_chunking(text, max_tokens=384):
    words = text.split()
    total_words = len(words)
    if total_words <= max_tokens:
        return text
    else:
        first_part = words[:int(total_words * 0.3)]
        last_part = words[-int(total_words * 0.6):]
        return " ".join(first_part + last_part)

In [ ]:
df['cleaned_text'] = df['text'].apply(clean_legal_text)
df['processed_text'] = df['cleaned_text'].apply(legal_chunking)

In [ ]:
model_name = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    inputs = tokenizer(examples["processed_text"], truncation=True, padding="max_length", max_length=384)
    targets = tokenizer(examples["summary"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
dataset = Dataset.from_pandas(df[['processed_text', 'summary']])
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/legal-pegasus-final-69",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    learning_rate=3e-5,
    warmup_steps=100,
    logging_steps=50,
    save_steps=200,
    report_to="none",
)

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)
print("Starting training...")
trainer.train()

trainer.save_model()
tokenizer.save_pretrained("/content/drive/MyDrive/legal-pegasus-final-69")

print("Training completed! Model saved to Google Drive.")

Starting training...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,3.881300
100,3.487300
150,2.706100
200,2.681200
250,2.338600
300,2.092400
350,2.182200
400,1.937900
450,2.097800
500,1.801100


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pi

Training completed! Model saved to Google Drive.


In [ ]:
import os

model_path = "/content/drive/MyDrive/legal-pegasus-final-69"
print("Files in Google Drive model folder:")
if os.path.exists(model_path):
    for file in os.listdir(model_path):
        print(f"- {file}")
else:
    print("Folder doesn't exist!")

print(f"\nFull path: {model_path}")

Files in Google Drive model folder:
- checkpoint-200
- checkpoint-400
- checkpoint-600
- checkpoint-680
- config.json
- generation_config.json
- model.safetensors
- training_args.bin
- tokenizer_config.json
- special_tokens_map.json
- spiece.model

Full path: /content/drive/MyDrive/legal-pegasus-final-69


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import re

model_path = "/content/drive/MyDrive/legal-pegasus-final-69/checkpoint-680"
tokenizer_path = "/content/drive/MyDrive/legal-pegasus-final-69"

model = PegasusForConditionalGeneration.from_pretrained(model_path)
tokenizer = PegasusTokenizer.from_pretrained(tokenizer_path)

print("Model loaded! Testing with faster generation...")

def summarize_legal_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    if len(words) > 200:
        first_part = words[:int(len(words) * 0.3)]
        last_part = words[-int(len(words) * 0.6):]
        text = " ".join(first_part + last_part)

    inputs = tokenizer(text, return_tensors="pt", max_length=256, truncation=True)

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=80,
        min_length=20,
        num_beams=2,
        early_stopping=True,
        do_sample=False,
        length_penalty=1.0
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


test_text = "A Promissory Note is a simpler, negotiable instrument, essentially an unconditional promise to pay, often used for short-term or informal loans.DEMAND PROMISSORY NOTEDated: 1st February 2026Place: BengaluruRs. 2,00,000/- (Rupees Two Lakhs Only)ON DEMAND, I/We, Ms. Riya Das, D/o Mr. A. Das, residing at [Address], promise to pay Mr. Sameer Reddy, S/o Mr. K. Reddy, residing at [Address], or order, the sum of Rupees Two Lakhs Only, together with interest at the rate of 9% per annum, compounding monthly, for value received in Cash.I/We waive the requirement of presentment for payment, notice of non-payment, protest, and notice of protest.(To be affixed with Revenue Stamp and signed across it by the Borrower)Borrower's SignatureMs. Riya Das"
print("Generating summary...")
result = summarize_legal_text(test_text)
print("SUMMARY:", result)

Model loaded! Testing with faster generation...
Generating summary...
SUMMARY: A Promissory Note is a simple, negotiable instrument, essentially an unconditional promise to pay. It is used for short-term or informal loans.<n>The Note is a simple, negotiable instrument, essentially an unconditional promise to pay. It is used for short-term or informal loans.
